In [80]:
import requests
from bs4 import BeautifulSoup
# Make a request to the website
r = requests.get('https://www.bbc.com')
# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')
# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':"module__content"}).find_all('a'):
    i['href'] = "https://www.bbc.com"+i['href'] + '?page=all'
    link.append(i['href'])
new_link = []
for i in range(0, len(link), 3):
    new_link.append(link[i])
print(new_link)
del new_link[3:4]
print(new_link)
# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in new_link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'ssrcss-7uxr49-RichTextContainer e5tfeyi1'}).find_all('p'):
        sen.append(i.text)

    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))
    print(documents)

['https://www.bbc.com/news/world-asia-china-64102176?page=all', 'https://www.bbc.com/news/world-us-canada-64099642?page=all', 'https://www.bbc.com/news/world-europe-64101437?page=all', 'https://www.bbc.com/sport/live/football/63641454?page=all', 'https://www.bbc.com/news/world-europe-64102180?page=all']
['https://www.bbc.com/news/world-asia-china-64102176?page=all', 'https://www.bbc.com/news/world-us-canada-64099642?page=all', 'https://www.bbc.com/news/world-europe-64101437?page=all', 'https://www.bbc.com/news/world-europe-64102180?page=all']
['Chinese people have rushed to book overseas travel after Beijing announced it would reopen its borders next month.']
['Chinese people have rushed to book overseas travel after Beijing announced it would reopen its borders next month.', 'The death toll in the New York city of Buffalo has risen to 28, with thousands still without power amid a monster winter storm that has battered North America. ']
['Chinese people have rushed to book overseas tra

In [81]:
import re
import string
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

In [82]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(documents_clean)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())
df

,0,1,2,3
after,0.193214,0.000000,0.173495,0.000000
agreed,0.000000,0.000000,0.000000,0.231375
america,0.000000,0.197002,0.000000,0.000000
amid,0.000000,0.197002,0.000000,0.000000
an,0.000000,0.000000,0.220056,0.000000
...,...,...,...,...
winter,0.000000,0.197002,0.000000,0.000000
with,0.000000,0.155319,0.000000,0.182419
without,0.000000,0.197002,0.000000,0.000000
would,0.245067,0.000000,0.000000,0.000000


In [83]:
def get_similar_articles(q, df):
  import numpy as np
  print("query:", q)
  print("The following are articles with the highest cosine similarity values: ")
    # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
    # Calculate the similarity
  for i in range(10):
     sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
    
    # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Nilai Similaritas:", v)
      print(documents_clean[k])        
      print()
  # Add The Query


q1 = input(str)
  # Call the function
get_similar_articles(q1, df)

query: a
Berikut artikel dengan nilai cosine similarity tertinggi: 


KeyError: 4